In [1]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, PReLU, Flatten, Softmax
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow.compat.v1 as tf
import tqdm
import numpy as np
import cv2
import random

In [2]:
def preprocess_image(image_path, size = None):
    if size is not None:
        img = load_img(image_path, target_size=size)
    else:
        img = load_img(image_path)
    # scale image to [0,1]
    img = img_to_array(img) / 255
    # to numpy array
    return np.array(img, dtype='float32')

In [3]:
def create_scaled_batch(image, target_shape, stride):
    (ih, iw, _) = image.shape
    (h, w) = target_shape
    (sx, sy) = stride
    images = np.empty((1,w,h,3), dtype='float32')
    coords = []
    for y in range(h, ih, sy):
        for x in range(w, iw, sx):
            new_img = image[y-h:y, x-w:x, :]
            new_img = new_img.reshape(1, *new_img.shape)
            images = np.append(images, new_img, axis=0)
            coords.append(np.array([x-w,y-h, x, y]))
    return images, np.array(coords)

In [4]:
def compute_scale_pyramid(m, min_layer):
    scales = []
    factor_count = 0

    while min_layer >= 12:
        scales += [m * np.power(SCALE_FACTOR, factor_count)]
        min_layer *= SCALE_FACTOR
        factor_count += 1
    
    return scales

In [5]:
def resizing_scales(shape):
    if len(shape) is not 3:
        logging.fatal('Invalid argument')
    (h, w, ch) = shape
    
    # initialize scale
    prev_scale = 1.0

    if min(w,h) > 500:
        prev_scale = 500./min(w, h)
    elif max(w,h) < 500:
        prev_scale = 500./max(w, h)

    w = int(w * prev_scale)
    h = int(h * prev_scale)

    # multi scale
    scales = []
    factor = 0.709
    factor_count = 0
    min_layer = min(h,w)
    while min_layer >= 12:
        scales += [prev_scale * pow(factor, factor_count)]
        min_layer *= factor
        factor_count += 1
    
    return scales

In [6]:
def faces_PNet(cls_prob, roi, coords, scale, width, height, thres):
    good = np.where(cls_prob>=thres)[0]
    rectangles = (roi[good] * 12. + coords[good]) * scale
    squares = rect2square(rectangles)
    probs = cls_prob[good]
    pick = []
    for i in range(len(rectangles)):
        x1 = int(max(0     ,squares[i][0]))
        y1 = int(max(0     ,squares[i][1]))
        x2 = int(min(width ,squares[i][2]))
        y2 = int(min(height,squares[i][3]))
        sc = probs[i]
        if x2>x1 and y2>y1:
            pick.append([x1,y1,x2,y2,sc])
    return NMS(pick,0.3,'iou')


In [7]:
def NMS(rectangles,threshold,type):
    if len(rectangles)==0:
        return rectangles
    boxes = np.array(rectangles)
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    s  = boxes[:,4]
    area = np.multiply(x2-x1+1, y2-y1+1)
    I = np.array(s.argsort())
    pick = []
    while len(I)>0:
        xx1 = np.maximum(x1[I[-1]], x1[I[0:-1]]) #I[-1] have hightest prob score, I[0:-1]->others
        yy1 = np.maximum(y1[I[-1]], y1[I[0:-1]])
        xx2 = np.minimum(x2[I[-1]], x2[I[0:-1]])
        yy2 = np.minimum(y2[I[-1]], y2[I[0:-1]])
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        if type == 'iom':
            o = inter / np.minimum(area[I[-1]], area[I[0:-1]])
        else:
            o = inter / (area[I[-1]] + area[I[0:-1]] - inter)
        pick.append(I[-1])
        I = I[np.where(o<=threshold)[0]]
    result_rectangle = boxes[pick].tolist()
    return result_rectangle

In [8]:
def rect2square(rectangles):
    w = rectangles[:,2] - rectangles[:,0]
    h = rectangles[:,3] - rectangles[:,1]
    l = np.maximum(w,h).T
    rectangles[:,0] = rectangles[:,0] + w*0.5 - l*0.5
    rectangles[:,1] = rectangles[:,1] + h*0.5 - l*0.5 
    rectangles[:,2:4] = rectangles[:,0:2] + np.repeat([l], 2, axis = 0).T 
    return rectangles

In [9]:
def filter_face_rnet(cls_prob,roi,rectangles,width,height,threshold):
    prob = cls_prob[:,0]
    pick = np.where(prob>=threshold)
    rectangles = np.array(rectangles)
    x1  = rectangles[pick,0]
    y1  = rectangles[pick,1]
    x2  = rectangles[pick,2]
    y2  = rectangles[pick,3]
    sc  = np.array([prob[pick]]).T
    dx1 = roi[pick,0]
    dx2 = roi[pick,1]
    dx3 = roi[pick,2]
    dx4 = roi[pick,3]
    w   = x2-x1
    h   = y2-y1
    x1  = np.array([(x1+dx1*w)[0]]).T
    y1  = np.array([(y1+dx2*h)[0]]).T
    x2  = np.array([(x2+dx3*w)[0]]).T
    y2  = np.array([(y2+dx4*h)[0]]).T
    rectangles = np.concatenate((x1,y1,x2,y2,sc),axis=1)
    rectangles = rect2square(rectangles)
    pick = []
    for i in range(len(rectangles)):
        x1 = int(max(0     ,rectangles[i][0]))
        y1 = int(max(0     ,rectangles[i][1]))
        x2 = int(min(width ,rectangles[i][2]))
        y2 = int(min(height,rectangles[i][3]))
        sc = rectangles[i][4]
        if x2>x1 and y2>y1:
            pick.append([x1,y1,x2,y2,sc])
    return NMS(pick,0.3,'iou')

In [10]:
def filter_face_onet(cls_prob,roi,rectangles,width,height,threshold):
    prob = cls_prob[:,0]
    pick = np.where(prob>=threshold)
    rectangles = np.array(rectangles)
    x1  = rectangles[pick,0]
    y1  = rectangles[pick,1]
    x2  = rectangles[pick,2]
    y2  = rectangles[pick,3]
    sc  = np.array([prob[pick]]).T
    dx1 = roi[pick,0]
    dx2 = roi[pick,1]
    dx3 = roi[pick,2]
    dx4 = roi[pick,3]
    w   = x2-x1
    h   = y2-y1
    x1  = np.array([(x1+dx1*w)[0]]).T
    y1  = np.array([(y1+dx2*h)[0]]).T
    x2  = np.array([(x2+dx3*w)[0]]).T
    y2  = np.array([(y2+dx4*h)[0]]).T
    rectangles=np.concatenate((x1,y1,x2,y2,sc),axis=1)
    pick = []
    for i in range(len(rectangles)):
        x1 = int(max(0     ,rectangles[i][0]))
        y1 = int(max(0     ,rectangles[i][1]))
        x2 = int(min(width ,rectangles[i][2]))
        y2 = int(min(height,rectangles[i][3]))
        if x2>x1 and y2>y1:
            pick.append([x1,y1,x2,y2, rectangles[i][4]])
    return NMS(pick,0.3,'iom')

In [11]:
image_path = 'Sofia.jpeg'
pweights_path = './models/pnet.h5'
rweights_path = './models/rnet.h5'
oweights_path = './models/onet.h5'

In [12]:
pnet = load_model(pweights_path)
rnet = load_model(rweights_path)
onet = load_model(oweights_path)

In [13]:
pnet.summary()

Model: "PNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
PNet_Input (InputLayer)         [(None, 12, 12, 3)]  0                                            
__________________________________________________________________________________________________
PNet_CONV1 (Conv2D)             (None, 10, 10, 10)   280         PNet_Input[0][0]                 
__________________________________________________________________________________________________
PNet_PRELU1 (PReLU)             (None, 10, 10, 10)   10          PNet_CONV1[0][0]                 
__________________________________________________________________________________________________
PNet_MAXPOOL1 (MaxPooling2D)    (None, 5, 5, 10)     0           PNet_PRELU1[0][0]                
_______________________________________________________________________________________________

In [14]:
MIN_FACE_SIZE = 40
SCALE_FACTOR = 0.709

In [15]:
image = preprocess_image(image_path)
print(image.shape)

(orig_h, orig_w, ch) = image.shape
scales = resizing_scales(image.shape)

In [22]:
pnet_out = []
coordinates = []
for scale in tqdm.tqdm(scales):
    new_h = int(orig_h * scale)
    new_w = int(orig_w * scale)
    scaled_img = preprocess_image(image_path, (new_h, new_w))
    out = pnet.predict(np.expand_dims(scaled_img, 0))
    pnet_out.append(out)

100%|██████████| 11/11 [00:03<00:00,  3.19it/s]


In [23]:
output = pnet_out[0]

In [30]:
output[1].shape

(1, 328, 245, 4)

In [28]:
np.transpose(output[0], (0,2,1,3)).shape

(1, 245, 328, 1)

In [ ]:
rectangles = []
for output, scale, coords in zip(pnet_out, scales, coordinates):
    size = output[0].shape[0]
    cls_prob = output[0].reshape(size, -1)
    roi_prob = output[1].reshape(size, -1)
    rectangle = faces_PNet(cls_prob, roi_prob, coords, 1 / scale, orig_w, orig_h, 0.7)
    rectangles.extend(rectangle)
rectangles = NMS(rectangles, 0.7, 'iou')


In [ ]:
len(rectangles)

In [ ]:
crop_number = 0
out = []
predict_24_batch = []
for rectangle in rectangles:
    coords = [int(i) for i in rectangle]
    crop_img = image[coords[1]:coords[3], coords[0]:coords[2]]
    scale_img = cv2.resize(crop_img, (24,24))
    predict_24_batch.append(scale_img)
    crop_img += 1

predict_24_batch = np.array(predict_24_batch)
out = rnet.predict(predict_24_batch)

In [ ]:
cls_prob = np.array(out[0])
roi_prob = np.array(out[1])

In [ ]:
rectangles = filter_face_rnet(cls_prob, roi_prob, rectangles, orig_w, orig_h, 0.7)

In [ ]:
len(rectangles)

In [ ]:
crop_number = 0
predict_batch = []
for rectangle in rectangles:
    coords = [int(i) for i in rectangle]
    crop_img = image[coords[1]:coords[3], coords[0]:coords[2]]
    scale_img = cv2.resize(crop_img, (48, 48))
    predict_batch.append(scale_img)
    crop_number += 1

predict_batch = np.array(predict_batch)
output = onet.predict(predict_batch)

In [ ]:
cls_prob = output[0]
roi_prob = output[1]

In [ ]:
rectangles = filter_face_onet(cls_prob, roi_prob, rectangles, orig_w, orig_h, 0.9)

In [ ]:
rectangles

### PUT IT ALL TOGETHER

In [ ]:
img = cv2.imread(image_path)

In [ ]:
draw = img.copy()
for rectangle in rectangles:
    if rectangle is not None:
        rect = [int(i) for i in rectangle]
        W = -rect[0] + rect[2]
        H = -rect[1] + rect[3]
        paddingH = 0.01 * W
        paddingW = 0.01 * H
        crop_img = img[int(rect[1]+paddingH):int(rect[3]-paddingH), int(rect[0]-paddingW):int(rect[2]+paddingW)]
        crop_img = cv2.cvtColor(crop_img, cv2.COLOR_RGB2GRAY)
        if crop_img is None:
            continue
        if crop_img.shape[0] < 0 or crop_img.shape[1] < 0:
            continue
        cv2.rectangle(draw, (rect[0], rect[1], rect[2], rect[3]), (255, 0, 0), 4)

In [ ]:
cv2.imwrite('SOFIA_FACE.jpeg', draw)